In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.5 MB/s eta 0:00:00


In [3]:
pip install transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00


Парсинг библиоглобус

In [ ]:
#Создаем пустые списки для сохранения данных
data = []
cur_item = []

#Проходим циклом по всем страницам раздела на сайте Библио-глобуса
for i in range(1, 137):
    print(i)

    url1 = f'https://www.biblio-globus.ru/catalog/category?id=226&page={i}&sort=0&instock=&isdiscount='

    #Берем страницу раздела по url
    r1 = requests.get(url1)

    #Превращаем страницу в объект супа
    soup1 = BeautifulSoup(r1.text, 'lxml')

    #Находим все книги на странице
    s1 = soup1.find_all(class_="product")

    #В цикле пробегаем по всем найденным книгам
    for item in s1:

        #Создаем пустой список для сохранения данных о книге
        cur_item = []

        #Из объекта супа извлекаем ссылку на текущую книгу
        s2 = item.find('h3').find('a').get('href')

        #Соединяем ссылку и url сайта
        url2 = 'https://www.biblio-globus.ru' + s2

        #Сохраняем url книги
        cur_item.append(url2)

        #Получае страницу книги
        r2 = requests.get(url2)

        #Превращаем страницу в объект супа
        soup2 = BeautifulSoup(r2.text, 'lxml')

        #Сохраняем url обложки
        t4 = soup2.find('meta', attrs={'property': "og:image"})
        cur_item.append(t4['content'])

        #Сохраняем данные об авторе
        t2 = soup2.find('meta', attrs={'property': "og:book:author"})
        cur_item.append(t2['content'])

        #Сохраняем название книги
        t1 = soup2.find('meta', attrs={'property': 'og:title'})
        cur_item.append(t1['content'])

        #Сохраняем аннотацию к книге
        t3 = soup2.find(id="module").find(id="collapseExample").contents[0].strip()
        cur_item.append(t3)

        #Добавляем все данные в список списков питона
        data.append(cur_item)

#Создаем из списка списков датафрейм с заданными названиями колонок
columns = ['page_url', 'image_url', 'author', 'title', 'annotation']
df = pd.DataFrame(data, columns=columns)

#Сохраняем датафрейм в csv файл
df.to_csv('classic.csv', index=False)

In [ ]:
bookurl = requests.get(f'https://www.biblio-globus.ru/catalog/category?id=611b2&page={i}&sort=0&instock=&isdiscount=')

In [ ]:
url_list = []
for id in range(1,122):
  i = id
  bookurl = requests.get(f'https://www.biblio-globus.ru/catalog/category?id=6112&page={i}&sort=0&instock=&isdiscount=')
  a = BeautifulSoup(bookurl.text, 'html.parser')
  all_page_songs = a.find_all('a', attrs = {'class' : 'img_link'})
  for item in all_page_songs:
    item_href = 'https://www.biblio-globus.ru' + item.get('href')

    url_list.append(item_href)

In [ ]:
title_names = []
images_urls = []
anotation_list = []
authors = []
url_detect = []
for url in url_list:
  get_url = requests.get(url)
  a = BeautifulSoup(get_url.text, 'html.parser')
  try:
    titles = a.find('h1',).text
    images = a.find('img', attrs = {'class' : 'product_photo zoomIn img-fluid'}).get('src')
    author = a.find('a', attrs = {'class' : 'scroll-to'}).text.strip()
    description = a.find('div', {'class': 'collapse', 'id': 'collapseExample'}).text.strip()
    characteristics = a.find('h3', text='Характеристики')
    if characteristics:
      next_sibling = characteristics.find_next_sibling()
      if next_sibling:
          description = description.split(characteristics.text, 1)[0].strip()
    if author == '' or titles == '' or description == '' or images == '':
      print(f'{url}')
      url_detect.append(url)
    else:

      images_urls.append(images)
      authors.append(author)
      anotation_list.append(description)
      title_names.append(titles)
  except (requests.RequestException, ValueError, AttributeError) as e:
        print(f"Ошибка {url}")

for url in url_detect:
  url_list.remove(url)

In [ ]:
url_list.remove('https://www.biblio-globus.ru/product/10631772')

In [ ]:
df = pd.DataFrame({
    'page_url': url_list,
    'image_url': images_urls,
    'author': authors,
    'title': title_names,
    'annotation': anotation_list
})

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [ ]:
df = pd.read_csv('/content/ezo_classic_sovr_proza_merged_200.csv')

In [ ]:
len(df['annotation'])

5818

In [ ]:
df['embeddings'] = df['annotation'].apply(lambda x: embed_bert_cls(x, model, tokenizer))

# Сохраняем результат в файл формата txt
with open('embeddings.txt', 'w') as file:
    for emb in df['embeddings']:
        file.write(' '.join(map(str, emb)) + '\n')

In [ ]:
len(df['embeddings'])

5818

In [ ]:
with open('embeddings.txt', 'r') as file:
    embeddings_list = [list(map(float, line.split())) for line in file.readlines()]

In [5]:
with open('embeddingsbooks.txt', 'r') as file:
        embeddings_list = [list(map(float, line.split())) for line in file.readlines()]


In [8]:
embeddings_list_np = np.array(embeddings_list)

In [12]:
embeddings_list_np.shape[1]

312

Применение Faiss

In [13]:
d = embeddings_list_np.shape[1]

In [17]:
import faiss

ModuleNotFoundError: ignored

In [14]:
index = faiss.IndexFlatL2(d)

NameError: ignored

In [ ]:
index.add(embeddings_list_np)

In [ ]:
faiss.write_index(index, "faiss.index")

Объедение разных спаршенных разделов в один датафрейм

In [19]:
 a = pd.read_csv('/content/ezo_classic_sovr_proza_merged_200.csv')

In [21]:
c = pd.read_csv('/content/data (2).csv')

In [25]:
df_new = pd.concat([a, c], ignore_index=True)

In [29]:
df_final = df_new[df_new['annotation'].apply(lambda x: len(str(x)) >= 200)]

In [33]:
df_final['embeddings'] = df_final['annotation'].apply(lambda x: embed_bert_cls(x, model, tokenizer))

# Сохраняем результат в файл формата txt
with open('embeddings.txt', 'w') as file:
    for emb in df_final['embeddings']:
        file.write(' '.join(map(str, emb)) + '\n')

<ipython-input-33-39631c6fe045>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['embeddings'] = df_final['annotation'].apply(lambda x: embed_bert_cls(x, model, tokenizer))
